In [29]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json

In [30]:
def fix_JSON(json_message=None):
    result = None
    try:        
        result = json.loads(json_message)
    except Exception as e:      
        # Find the offending character index:
        idx_to_replace = int(e.message.split(' ')[-1].replace(')',''))      
        # Remove the offending character:
        json_message = list(json_message)
        json_message[idx_to_replace] = ' '
        new_message = ''.join(json_message)     
        return fix_JSON(json_message=new_message)
    return result

In [31]:
def clean_text(textdata):
    soup = BeautifulSoup(textdata)
    for script in soup(["script", "style"]):
        script.extract()
    text = soup.get_text()
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return (text.encode('utf-8'))

In [32]:
explainers = BeautifulSoup(requests.get("http://theconversation.com/au/factcheck").text, "lxml")
headlines = explainers.find_all('h2')
results = list()

In [33]:
for headline in headlines:
    thisline = dict()
    try:
        link = "http://theconversation.com" + headline.a['href']
        thisline['url'] = link
        thisline['headline'] = headline.text
        thisline['content'] = requests.get(link).text
        soup = BeautifulSoup(thisline['content'], "lxml")
        thisline['link'] = link
        meta = soup.find(attrs={'name' : "news_keywords"})
        thisline['keywords'] = meta['content']
        thisline['source'] = "The Conversation"
    except:
        print("That went horribly wrong")
    results.append(thisline)

That went horribly wrong
That went horribly wrong


In [6]:
abcexplainers = BeautifulSoup(requests.get('http://www.abc.net.au/news/factcheck/zombies/').text, "lxml")

links = abcexplainers.find_all("a")

In [ ]:
links

In [7]:
links = [link for link in links if "Fact Check zombie" in link.text]

In [ ]:
thisline = list()

In [20]:
links[0]

<a href="/news/2016-06-15/fact-check-zombie-bill-shorten-repeats-misleading-claim/7509112">Fact Check zombie: Bill Shorten repeats misleading Tony Abbott and Joe Hockey claim on changing the GST</a>

In [27]:
for link in links:
    abcid = link['href'].split('/')[-1]
    url = "https://content-api-govhack.abc-prod.net.au/v1/" + abcid
    data = requests.get(url).text
    try:
        jsondoc = json.loads(data)
    except:
        print("Bad JSON")
        print (data)
        continue
    try:
        article = dict()
        article['url'] = url
        article['headline'] = jsondoc['shortTeaserTextPlain']
        article['content'] = jsondoc['textPlain']
        article['link'] = url
        article['keywords'] = jsondoc['keywords']
        article['source'] = "ABC Fact Check"
        thisline.append(article)
    except:
        print (jsondoc.keys())

Bad JSON
{
  "id" : 7509112,
  "docType" : "Article",
  "title" : "Fact Check zombie: Bill Shorten repeats misleading Tony Abbott and Joe Hockey claim on changing the GST",
  "shortTeaserTitle" : "Fact Check zombie: GST on tampons",
  "teaserTitle" : "Fact Check zombie: Bill Shorten repeats misleading Tony Abbott and Joe Hockey claim on changing the GST",
  "shortTeaserTextPlain" : "Zombie alert! Bill Shorten repeats misleading Tony Abbott and Joe Hockey claim on changing the GST.",
  "teaserTextPlain" : "Opposition Leader Bill Shorten has confirmed that Labor will not be scrapping the GST on tampons if elected, despite a concerted campaign from the public to do so — a backflip from a previous position.\n\n\"In terms of the GST on tampons, in an ideal world we would scrap that. But in order to do that you have to get the concurrence of state premiers,\" Mr Shorten told a press conference on June 10, 2016.\n\nFact Check has already tested claims on whether state assent would be needed t

In [22]:
test = requests.get("https://content-api-govhack.abc-prod.net.au/v1/7509112")

In [24]:
test.text

'{\n  "id" : 7509112,\n  "docType" : "Article",\n  "title" : "Fact Check zombie: Bill Shorten repeats misleading Tony Abbott and Joe Hockey claim on changing the GST",\n  "shortTeaserTitle" : "Fact Check zombie: GST on tampons",\n  "teaserTitle" : "Fact Check zombie: Bill Shorten repeats misleading Tony Abbott and Joe Hockey claim on changing the GST",\n  "shortTeaserTextPlain" : "Zombie alert! Bill Shorten repeats misleading Tony Abbott and Joe Hockey claim on changing the GST.",\n  "teaserTextPlain" : "Opposition Leader Bill Shorten has confirmed that Labor will not be scrapping the GST on tampons if elected, despite a concerted campaign from the public to do so — a backflip from a previous position.\\n\\n\\"In terms of the GST on tampons, in an ideal world we would scrap that. But in order to do that you have to get the concurrence of state premiers,\\" Mr Shorten told a press conference on June 10, 2016.\\n\\nFact Check has already tested claims on whether state assent would be nee

In [ ]:
results.extend(thisline)

In [ ]:
thisline[1]

In [16]:
thisline[1]['textPlain']

KeyError: 'textPlain'

In [34]:
df = pd.DataFrame(results)

In [40]:
df = df[['link','source','keywords','headline','content']]

In [41]:
df['content'] = df['content'].fillna(" ")

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [42]:
df['content'] = df['content'].apply(clean_text)

C:\Anaconda3\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [43]:
df['textPlain'] = df['textPlain'].fillna(df['content'])

KeyError: 'textPlain'

In [ ]:
newframe = df[pd.notnull(df['text'])]

In [ ]:
newframe

In [ ]:
df.to_pickle('explainers.pickle')

In [44]:
content = pd.concat()

,link,source,keywords,headline,content
0,http://theconversation.com/election-factcheck-...,The Conversation,"Asylum seekers, Election factcheck, Election f...",Election FactCheck: Does the government spend ...,"b""Election FactCheck: Does the government spen..."
1,http://theconversation.com/election-factcheck-...,The Conversation,"Coalition, Election factcheck 2016, Federal el...",Has the Coalition invested an average of $5 bi...,"b""Election FactCheck: Has the Coalition invest..."
2,http://theconversation.com/election-factcheck-...,The Conversation,"Diagnostic tests, Election factcheck 2016, Hea...",Has the Coalition cut bulk-billing for patholo...,"b""Election FactCheck: has the Coalition cut bu..."
3,http://theconversation.com/election-factcheck-...,The Conversation,"Election factcheck, Employment, Federal electi...","Have 300,000 new jobs been created in the last...","b'Election FactCheck: Have 300,000 new jobs be..."
4,http://theconversation.com/election-factcheck-...,The Conversation,Australian Charities and not-for-profit Commis...,Is the Australian Sex Party right about religi...,"b""Election FactCheck: is the Australian Sex Pa..."
5,http://theconversation.com/election-factcheck-...,The Conversation,"Capital gains tax, Capital gains tax concessio...",Does the government spend more on negative gea...,"b""Election FactCheck Q&A: does the government ..."
6,http://theconversation.com/election-factcheck-...,The Conversation,"Climate change, Climate science, Election fact...","Are larger, more frequent storms predicted due...","b'Election FactCheck: are larger, more frequen..."
7,http://theconversation.com/election-factcheck-...,The Conversation,"Bill Shorten, Election factcheck 2016, Employm...","Have 50,000 full-time jobs been lost this year...","b""Election FactCheck: have 50,000 full-time jo..."
8,http://theconversation.com/election-factcheck-...,The Conversation,"Election factcheck 2016, Federal election 2016...",Is it true no solely managed Commonwealth fish...,"b""Election FactCheck: is it true no solely man..."
9,http://theconversation.com/election-factcheck-...,The Conversation,"Bill Shorten, Debt, Election factcheck 2016, F...",Has $100 billion been added to Australia’s nat...,"b""Election FactCheck: has $100 billion been ad..."


In [ ]:
soup = BeautifulSoup(somecontent)